In [ ]:
import sys
sys.path.append("./..")
from config import config

import argparse
import numpy as np
from tqdm import tqdm
from pathlib import Path
import os, sys
from glob import glob
import shutil
import json
import librosa
import torch
from transformers import AutoFeatureExtractor, ASTForAudioClassification, pipeline

In [ ]:
device = torch.device(config['cude_device'])   

feature_extractor = AutoFeatureExtractor.from_pretrained(config["event_classification_model"])
model = ASTForAudioClassification.from_pretrained(config["event_classification_model"])
model.to(device)
pass

In [ ]:
with open(config['json_path'], 'r') as fp:
    all_data = json.load(fp)

In [ ]:
audio_paths = glob(os.path.join(config['podcast']['path'], "*.wav"))
audio_paths.sort()
audio_paths = {os.path.basename(x).split('.')[0]:x for x in audio_paths}

In [ ]:
class ProcessPipeline(torch.utils.data.Dataset):
    def __init__(self, all_data, audios):
        print("Organizing the data")
        self.X = []
        self.segments = []
        self.audios = audios
        self.data = all_data
        for i, pod_name in enumerate(tqdm(list(all_data.keys()))):
            for seg_name, seg in all_data[pod_name].items():
                if 'speech_classification' in seg: continue
                if (seg['end']-seg['start']) > 500: continue
                
                self.X.append([seg['end']-seg['start'], pod_name, seg_name])
        
        self.X.sort(reverse=False)

        
        
    def __len__(self):
        return len(self.X)
        
        
    def __getitem__(self, index):
        """ get a video and its label """
        _, podcast, seg_name  = self.X[index]
        
        seg = self.data[podcast][seg_name]

        start = seg['start']
        end = seg['end']
        wav, sr = librosa.load(self.audios[podcast], offset=start, duration=end-start, sr=16000)
#         x = test_data_processing(wav, self.mean, self.std)
#         x = x.reshape((1, x.shape[0], x.shape[1]))
        
        return podcast, seg_name, wav


In [ ]:
def colate_fun(x):
    data = []
    lengths = []
    for sample in x:
        data.append([sample[0], sample[1], np.array(sample[2])])
        lengths.append(len(sample[2]))
        
    aud = np.zeros((len(x), max(lengths)))
    for i, a in enumerate(data):
        aud[i][:len(a[2])] = a[2]
    return data, aud, lengths

In [ ]:
len(all_data)

In [ ]:
process_data = ProcessPipeline(all_data, audio_paths)
process_loader = torch.utils.data.DataLoader(process_data, batch_size = 32, num_workers=8, pin_memory=True, shuffle = False,
                                              collate_fn=colate_fun)

In [ ]:
for info, auds, lengths in tqdm(process_loader):
    inputs = feature_extractor(auds, sampling_rate=16000, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
    
    predicted_class_ids = torch.argmax(logits, dim=-1)#.item()
    
    for i, idx, length in zip(info, predicted_class_ids, lengths):
        podcast, seg_name, x =  i
        predicted_labels = model.config.id2label[int(idx)] 
        all_data[podcast][seg_name]['speech_classification'] = predicted_labels

In [ ]:
with open(config['json_path'], 'w') as fp:
    json.dump(all_data, fp)